In [4]:
from splinter import Browser
from bs4 import BeautifulSoup
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [5]:
#set up executable path
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)



====== WebDriver manager ======
Current google-chrome version is 91.0.4472
Get LATEST driver version for 91.0.4472
Driver [/Users/jds/.wdm/drivers/chromedriver/mac64/91.0.4472.101/chromedriver] found in cache


## Scrape news from redplanet website

In [7]:
#setup beautiful soup for redplanetscience.com

#URL to be scraped
url = "https://www.redplanetscience.com"

#visit url
browser.visit(url)

#Grab the html 
news_html = browser.html

#create beautiful soup object with parser - parse in the html elements
soup = BeautifulSoup(news_html, "html.parser")

In [8]:
#find most recent news title
news_title = soup.find('div', class_="content_title").text

"NASA's Treasure Map for Water Ice on Mars"

In [9]:
#find paragraph text
news_p = soup.find('div', class_="article_teaser_body").text

'A new study identifies frozen water just below the Martian surface, where astronauts could easily dig it up.'

## Scrape photo from mars space image website

In [8]:
def featured_image_url():
    url="https://spaceimages-mars.com/"
    html = browser.visit(url)
    #locate the button and click it
    image_element = browser.find_by_tag("button")[1]
    image_element.click()
    #Parse with beautiful soup
    html = browser.html
    img_soup = BeautifulSoup(html, 'html.parser')
    #Select image src and add it to the url
    image_anchor = img_soup.find("img", class_="fancybox-image").get('src')
    featured_image_url = url + image_anchor
    return featured_image_url

In [9]:
featured_image_url()

'https://spaceimages-mars.com/image/featured/mars2.jpg'

# Mars Facts

In [98]:
def mars_facts():
    mars_table = pd.read_html('https://galaxyfacts-mars.com')[0]
    mars_table.columns = ['description', 'mars', 'earth']
    mars_table.set_index('description', inplace=True)
    mars_df = mars_table.to_html()
    return mars_df

In [99]:
mars_facts()

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>mars</th>\n      <th>earth</th>\n    </tr>\n    <tr>\n      <th>description</th>\n      <th></th>\n      <th></th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>Mars - Earth Comparison</th>\n      <td>Mars</td>\n      <td>Earth</td>\n    </tr>\n    <tr>\n      <th>Diameter:</th>\n      <td>6,779 km</td>\n      <td>12,742 km</td>\n    </tr>\n    <tr>\n      <th>Mass:</th>\n      <td>6.39 × 10^23 kg</td>\n      <td>5.97 × 10^24 kg</td>\n    </tr>\n    <tr>\n      <th>Moons:</th>\n      <td>2</td>\n      <td>1</td>\n    </tr>\n    <tr>\n      <th>Distance from Sun:</th>\n      <td>227,943,824 km</td>\n      <td>149,598,262 km</td>\n    </tr>\n    <tr>\n      <th>Length of Year:</th>\n      <td>687 Earth days</td>\n      <td>365.24 days</td>\n    </tr>\n    <tr>\n      <th>Temperature:</th>\n      <td>-87 to -5 °C</td>\n      <td>-88 to 58°C</td>\n    </tr>\n  </tbody>

## Scrape hemisphere data from mars hemispheres

In [14]:
def mars_hemispheres():
    #Visit mars hemispheres website
    url = 'https://marshemispheres.com/'
    browser.visit(url)

    hemisphere_image_links = []
    
    #get links for all hemispheres
    links = browser.find_by_css('a.product-item img')
    print(links)
    
    #click all links and get the href
    for i in range(len(links)):
        hemisphere_dict = {}
        
        browser.find_by_css('a.product-item img')[i].click()
        thumb_img = browser.links.find_by_text('Sample').first
        print(thumb_img)

        #hemisphere href
        hemisphere_dict['img_url'] = thumb_img['href']

        #hemisphere title
        hemisphere_dict['title'] = browser.find_by_css('h2.title').text

        hemisphere_image_links.append(hemisphere_dict)

        browser.back()
    return hemisphere_image_links

In [15]:
mars_hemispheres()

[{'img_url': 'https://marshemispheres.com/images/full.jpg',
  'title': 'Cerberus Hemisphere Enhanced'},
 {'img_url': 'https://marshemispheres.com/images/schiaparelli_enhanced-full.jpg',
  'title': 'Schiaparelli Hemisphere Enhanced'},
 {'img_url': 'https://marshemispheres.com/images/syrtis_major_enhanced-full.jpg',
  'title': 'Syrtis Major Hemisphere Enhanced'},
 {'img_url': 'https://marshemispheres.com/images/valles_marineris_enhanced-full.jpg',
  'title': 'Valles Marineris Hemisphere Enhanced'}]

# Combine all dicts

In [101]:
def scrape_all():
    executable_path = {'executable_path': ChromeDriverManager().install()}
    browser = Browser('chrome', **executable_path, headless=True)
    mars_data = {
        "news_title": news_title,
        "news_p": news_p,
        "featured_image": featured_image_url(browser),
        "mars_facts": mars_facts(),
        "hemispheres": mars_hemispheres(browser),
    }
    browser.quit()
    return(mars_data)